In [ ]:
##Opening Data And saving

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.

drive.mount('/content/drive/')
data_path = '/content/drive/Shareddrives/IR'
output_path = '/content/drive/Shareddrives/IR/outputs/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install config

In [ ]:
from google.colab import files
uploaded = files.upload()
for file in uploaded:
  print("file name : {} , length: {}".format(file,len(file)))

Saving myfunctions.py to myfunctions.py
file name : myfunctions.py , length: 14


In [ ]:
from myfunctions import *
from nltk import word_tokenize

In [ ]:
OUTPUT_FILENAME = "/data/tweets100_hydrated.json"

with open(data_path+OUTPUT_FILENAME, 'r') as f:
    lines = f.readlines()
    data = [json.loads(str_) for str_ in lines]

In [ ]:
df_tweets = pd.DataFrame.from_records(data)
df_tweets["retweeted_status"]=df_tweets["retweeted_status"].apply(lambda x: 0 if str(x)=="nan" else x)

df_original = df_tweets[df_tweets["retweeted_status"]==0]
#0: tweets are not RT
#1: tweets are RT
df_original.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,possibly_sensitive,extended_entities,withheld_in_countries
1,Thu Dec 17 13:47:46 +0000 2020,1339567967223250947,1339567967223250947,Oh yr eh ohi nhi aa jine corona kr k parliamen...,False,"[0, 157]","{'hashtags': [{'text': 'FarmersProtest', 'indi...","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,"{'id': 863024748590059521, 'id_str': '86302474...",None,None,None,None,0,True,1,7,False,False,en,1.339402e+18,1339402143866912770,"{'url': 'https://t.co/S8foPwBiKY', 'expanded':...",{'created_at': 'Thu Dec 17 02:48:50 +0000 2020...,False,NaN,NaN
4,Thu Dec 17 13:47:46 +0000 2020,1339567967613366272,1339567967613366272,@sambosnumerouno @MENnewsdesk @ccoxmenmedia Ye...,False,"[44, 322]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1.339567e+18,1339566783964045312,9.286508e+17,928650799349161984,sambosnumerouno,"{'id': 2831341487, 'id_str': '2831341487', 'na...",None,None,None,None,0,False,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Thu Dec 17 13:47:46 +0000 2020,1339567967680475136,1339567967680475136,How about not acknowledging Covid\nThe dangers...,False,"[0, 196]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/#!/download/ipad"" ...",NaN,None,NaN,None,None,"{'id': 802581205262696448, 'id_str': '80258120...",None,None,None,None,0,True,0,0,False,False,en,1.339376e+18,1339375759677411328,"{'url': 'https://t.co/ukUd8G4XQP', 'expanded':...",{'created_at': 'Thu Dec 17 01:04:00 +0000 2020...,False,NaN,NaN
8,Thu Dec 17 13:47:46 +0000 2020,1339567967718305794,1339567967718305794,fuck we really boutta be 5-9 😔 https://t.co/h8...,False,"[0, 30]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 1037829476267249664, 'id_str': '1037829...",None,None,None,None,0,True,0,1,False,False,en,1.339568e+18,1339567549768491012,"{'url': 'https://t.co/h8gTqdoBfm', 'expanded':...",{'created_at': 'Thu Dec 17 13:46:06 +0000 2020...,False,NaN,NaN
11,Thu Dec 17 13:47:46 +0000 2020,1339567968234201093,1339567968234201093,oh no 😂 https://t.co/op7jImeDRi,False,"[0, 7]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,"{'id': 1313181832700194818, 'id_str': '1313181...",None,None,None,None,0,True,0,0,False,False,en,1.339568e+18,1339567549768491012,"{'url': 'https://t.co/op7jImeDRi', 'expanded':...",{'created_at': 'Thu Dec 17 13:46:06 +0000 2020...,False,NaN,NaN


In [ ]:
start_time = time.time()
numDocuments = len(df_original)
index, tf, df, idf = create_index_tfidf(df_original, numDocuments)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))

Total time to create the index: 133.47 seconds


In [ ]:
def search_tf_idf(query, index,df):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''

    query=preprocess_tweet(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass


    ranked_docs = rankDocuments(query, docs, index, idf, tf) 

    docs=list(ranked_docs)
    output = []


    for i in docs:
      hashtag = []
      if df.loc[i]["entities"]["hashtags"]:
        for k in range(len(df.loc[i]["entities"]["hashtags"])):
          hashtag.append(df.loc[i]["entities"]["hashtags"][k]["text"])
      else:
        hashtag.append("No Hashtags")

      text = regex.sub('@[^\s]+', '', df.loc[i]["full_text"])
      text = regex.sub(r'http\S+', '', text)

    
      output.append([query,text, df.loc[i]["user"]["screen_name"], df.loc[i]["created_at"], hashtag, df.loc[i]["favorite_count"], df.loc[i]["retweet_count"],regex.findall('http\S+', df.loc[i]["full_text"])])

    
    
    return output

In [ ]:
queries = ["Obama", "Pfizer", "Trump", "Moderna", "Macron", "Germany", "Spain", "restaurant", "economy", "school"]
ranked_docsRQ2bawithout = []

for query in queries:
  new_ranking = search_tf_idf(query, index, df_original)
  for i in range(0,10):
    ranked_docsRQ2bawithout.append(new_ranking[i])

In [ ]:
df_RQ2Awithout = pd.DataFrame(ranked_docsRQ2bawithout, columns = ["query","Tweet", "Username", "Date", "Hashtags", "Likes", "Retweets", "URL"])

df_RQ2Awithout

,query,Tweet,Username,Date,Hashtags,Likes,Retweets,URL
0,[obama],Now do Reagan through Obama.,J_Dubya80,Thu Dec 17 14:01:56 +0000 2020,[No Hashtags],0,0,[https://t.co/QEd7LnMW3E]
1,[obama],Why would Trump trust Obama’s pandemic t...,FightandDebate,Thu Dec 17 13:53:28 +0000 2020,[No Hashtags],0,0,[]
2,[obama],But Obama wore a tan suit?,ssurovell,Thu Dec 17 13:53:57 +0000 2020,[No Hashtags],52,7,[https://t.co/wr6XbZvQXt]
3,[obama],Corona virus vaccine research was started an...,EntropyJimJohn,Thu Dec 17 14:14:07 +0000 2020,[No Hashtags],0,0,[]
4,[obama],GOP congressman who once shouted ‘you lie’ dur...,thomaskaine5,Thu Dec 17 14:12:41 +0000 2020,[No Hashtags],85,27,[https://t.co/2HVGDpakUR]
...,...,...,...,...,...,...,...,...
95,[school],CBD is helping school kids cope with the new ...,AFCaptainStraw,Thu Dec 17 14:24:49 +0000 2020,[No Hashtags],0,0,[https://t.co/WBYMTkV7Rw]
96,[school],CBD is helping school kids cope with the new n...,AFCaptainStraw,Thu Dec 17 14:24:36 +0000 2020,[No Hashtags],1,0,[https://t.co/WBYMTkV7Rw]
97,[school],#UnionStrong reminder that many #NYC school bu...,labor_against,Thu Dec 17 13:49:30 +0000 2020,"[UnionStrong, NYC, COVID19]",5,3,[https://t.co/h2qpUXmTSx]
98,[school],"But sure, open schools.",StoneFree924,Thu Dec 17 14:33:40 +0000 2020,[No Hashtags],0,0,[https://t.co/AuxP8IYzhl]


In [ ]:
df_RQ2Awithout.loc[:10]

,query,Tweet,Username,Date,Hashtags,Likes,Retweets,URL
0,[obama],Now do Reagan through Obama.,J_Dubya80,Thu Dec 17 14:01:56 +0000 2020,[No Hashtags],0,0,[https://t.co/QEd7LnMW3E]
1,[obama],Why would Trump trust Obama’s pandemic t...,FightandDebate,Thu Dec 17 13:53:28 +0000 2020,[No Hashtags],0,0,[]
2,[obama],But Obama wore a tan suit?,ssurovell,Thu Dec 17 13:53:57 +0000 2020,[No Hashtags],52,7,[https://t.co/wr6XbZvQXt]
3,[obama],Corona virus vaccine research was started an...,EntropyJimJohn,Thu Dec 17 14:14:07 +0000 2020,[No Hashtags],0,0,[]
4,[obama],GOP congressman who once shouted ‘you lie’ dur...,thomaskaine5,Thu Dec 17 14:12:41 +0000 2020,[No Hashtags],85,27,[https://t.co/2HVGDpakUR]
5,[obama],GOP congressman who once shouted ‘you lie’ dur...,santo_aol,Thu Dec 17 14:04:24 +0000 2020,[No Hashtags],1,0,[https://t.co/u60Y6YhDQ3]
6,[obama],Biden worked as VP under Obama and handle...,Cece57938158,Thu Dec 17 14:21:17 +0000 2020,[No Hashtags],0,0,[]
7,[obama],Has Obama taken the vaccine? I’ll take it if B...,Chels_Ayyeee,Thu Dec 17 14:19:23 +0000 2020,[No Hashtags],0,0,[]
8,[obama],Was just looking to see whether Obama pardo...,Playerinthgame,Thu Dec 17 14:32:36 +0000 2020,[No Hashtags],2,0,[]
9,[obama],These Trumpublkkkans would be the ones fi...,JonathanTrubow,Thu Dec 17 14:35:05 +0000 2020,[No Hashtags],2,0,[]


In [ ]:
df_RQ2Awithout.to_csv(output_path+"df_RQ2Awithout.csv", sep =";")

In [ ]:
def search_tf_idf_diversity(query, index,df):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''

    query=preprocess_tweet(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass


    ranked_docs = rankDocuments(query, docs, index, idf, tf) 

    docs=list(ranked_docs)
    output = []
    diversityList = []
    diversityDataframe =pd.DataFrame(columns = ["id", "diversity_score"])
    for i in docs:
      text = df.loc[i]["full_text"]
      words = word_tokenize(text)
      diversity_score = 1.0*len(set(words))/len(words)
      diversityList.append(diversity_score)

    diversityDataframe ["id"] = docs 
    diversityDataframe ["diversity_score"] = diversityList

    
    diversityDataframe = diversityDataframe.sort_values ("diversity_score", ascending = False, ignore_index = True)
  
    x= 0
    for i in diversityDataframe["id"].values:

      hashtag = []
      if df.loc[i]["entities"]["hashtags"]:
        for k in range(len(df.loc[i]["entities"]["hashtags"])):
          hashtag.append(df.loc[i]["entities"]["hashtags"][k]["text"])
      else:
        hashtag.append("No Hashtags")

      text = regex.sub('@[^\s]+', '', df.loc[i]["full_text"])
      text = regex.sub(r'http\S+', '', text)

    
      output.append([query,diversityDataframe.loc[x]["diversity_score"],text, df.loc[i]["user"]["screen_name"], df.loc[i]["created_at"], hashtag, 
                     df.loc[i]["favorite_count"], df.loc[i]["retweet_count"],regex.findall('http\S+', df.loc[i]["full_text"])])
      x = x+1
    
    
    return output

In [ ]:
queries = ["Obama", "Pfizer", "Trump", "Moderna", "Macron", "Germany", "Spain", "restaurant", "economy", "school"]
ranked_docsRQ2b = []

for query in queries:
  new_ranking = search_tf_idf_diversity(query, index, df_original)
  for i in range(0,10):
    ranked_docsRQ2b.append(new_ranking[i])

In [ ]:
df_RQ2A_DiversityRanking = pd.DataFrame(ranked_docsRQ2b, columns = ["query", "Diversity Score","Tweet", "Username", "Date", "Hashtags", "Likes", "Retweets", "URL"])

df_RQ2A_DiversityRanking

,query,Diversity Score,Tweet,Username,Date,Hashtags,Likes,Retweets,URL
0,[obama],1.00000,Now do Reagan through Obama.,J_Dubya80,Thu Dec 17 14:01:56 +0000 2020,[No Hashtags],0,0,[https://t.co/QEd7LnMW3E]
1,[obama],1.00000,GOP congressman who once shouted ‘you lie’ dur...,thomaskaine5,Thu Dec 17 14:12:41 +0000 2020,[No Hashtags],85,27,[https://t.co/2HVGDpakUR]
2,[obama],1.00000,But Obama wore a tan suit?,ssurovell,Thu Dec 17 13:53:57 +0000 2020,[No Hashtags],52,7,[https://t.co/wr6XbZvQXt]
3,[obama],1.00000,GOP congressman who once shouted ‘you lie’ dur...,santo_aol,Thu Dec 17 14:04:24 +0000 2020,[No Hashtags],1,0,[https://t.co/u60Y6YhDQ3]
4,[obama],0.97619,Wilson became infamous after shouting out “You...,brian_warrick,Thu Dec 17 13:50:32 +0000 2020,[SmartNews],0,0,[https://t.co/REoEwYgsNl]
...,...,...,...,...,...,...,...,...,...
95,[school],1.00000,"After COVID-19 outbreak, San Ramon district ba...",mercnews,Thu Dec 17 14:01:41 +0000 2020,[No Hashtags],1,0,[https://t.co/SbNMIxCrZ1]
96,[school],1.00000,Millions in Colorado school funding at risk be...,denvereducation,Thu Dec 17 14:02:15 +0000 2020,[No Hashtags],0,0,[https://t.co/wL2KshO4nF]
97,[school],1.00000,England's secondary school pupils face stagger...,markwilliamsa1,Thu Dec 17 14:02:26 +0000 2020,[No Hashtags],0,0,[https://t.co/SQPoO8t5ZS]
98,[school],1.00000,All students and staff at Kerry school to be t...,TheSun_NI,Thu Dec 17 14:06:53 +0000 2020,[No Hashtags],0,0,[https://t.co/YEaUPR1nXO]


In [ ]:
df_RQ2A_DiversityRanking.to_csv(output_path+"df_RQ2A_DiversityRanking.csv", sep =";")